In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [31]:
# Download the data using yfinance.
data = yf.download('GC=F', start='2017-01-01', end='2019-12-01')
data.columns = data.columns.get_level_values(0)  
#data = data['Close'].values.reshape(-1, 1)

data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,1155.199951,1160.800049,1146.500000,1160.400024,1160.400024,52
2017-01-04,1156.300049,1165.000000,1156.300049,1163.800049,1163.800049,36
2017-01-05,1172.000000,1183.300049,1171.500000,1179.699951,1179.699951,500
2017-01-06,1176.500000,1176.500000,1171.900024,1171.900024,1171.900024,16
2017-01-09,1173.900024,1183.599976,1173.000000,1183.500000,1183.500000,62
...,...,...,...,...,...,...
2019-11-22,1468.599976,1468.599976,1463.099976,1463.099976,1463.099976,19
2019-11-25,1457.099976,1457.099976,1455.500000,1456.599976,1456.599976,14
2019-11-26,1459.800049,1459.800049,1459.800049,1459.800049,1459.800049,346362


In [10]:
news=pd.read_csv("C:/Users/akash/OneDrive/Documents/GitHub/MSDS_458_Group_N/Sentiment Analysis/Results_Sentiment_gold_Price.csv")

In [12]:
summary_df = news.groupby('Dates')['sentiment'].value_counts().unstack(fill_value=0)

In [14]:
summary_df = summary_df.reset_index()

In [16]:
summary_df['highest_sentiment'] = summary_df[['negative', 'neutral', 'positive']].idxmax(axis=1)


In [18]:
summary_df['Dates'] = pd.to_datetime(summary_df['Dates'], format='%d-%m-%Y')

In [19]:
summary_df = summary_df.sort_values(by='Dates')

In [ ]:
result_df = summary_df[['Dates', 'highest_sentiment']]
result_df = result_df.rename(columns={'Dates': 'Date'})

In [26]:
result_df['highest_sentiment'] = result_df['highest_sentiment'].replace({
    'positive': +1,
    'neutral': 0,
    'negative': -1
})

C:\Users\akash\AppData\Local\Temp\ipykernel_25196\2911225026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['highest_sentiment'] = result_df['highest_sentiment'].replace({


In [36]:
# Join the two DataFrames on 'Dates'
merged_df = pd.merge(data, result_df, on='Date', how='inner')  # Use 'inner', 'left', 'right', or 'outer' as needed

# Display the merged DataFrame
print(merged_df)



          Date         Open         High          Low        Close  \
0   2017-01-03  1155.199951  1160.800049  1146.500000  1160.400024   
1   2017-01-04  1156.300049  1165.000000  1156.300049  1163.800049   
2   2017-01-05  1172.000000  1183.300049  1171.500000  1179.699951   
3   2017-01-06  1176.500000  1176.500000  1171.900024  1171.900024   
4   2017-01-09  1173.900024  1183.599976  1173.000000  1183.500000   
..         ...          ...          ...          ...          ...   
506 2019-01-25  1297.400024  1297.400024  1297.400024  1297.400024   
507 2019-01-28  1300.800049  1307.099976  1298.400024  1302.400024   
508 2019-01-30  1310.400024  1323.400024  1308.099976  1309.900024   
509 2019-01-31  1320.000000  1325.400024  1316.699951  1319.699951   
510 2019-02-01  1320.300049  1320.699951  1315.400024  1316.900024   

       Adj Close  Volume  highest_sentiment  
0    1160.400024      52                  0  
1    1163.800049      36                 -1  
2    1179.699951     

In [37]:
merged_df.to_csv("final_gold_sentiment.csv", index=False)